# Example usage 

First we show how to use the package with paths. If you have three paths, one for train, one for test and one for your synthetic images, you are ready to go. 

In [5]:

from beyondfid.default_config import config 
from beyondfid.utils import update_config
from beyondfid.run import run

# decide what features to extract, run beyondfid -h for a full list, can be custom (see below)
feature_extractors="inception" 

 # decide what metrics you want to compute, run beyonfid -h for a full list
metrics ="fid,prdc"

# update the config
config = update_config(config, metrics=metrics, feature_extractors=feature_extractors) 

# manually set hyperparameters if necessary - full list is in beyondfid.default_config.py
config.metrics.prdc.nearest_k = 3

# path/to/synth can also be a generator function (see below)
results = run("/vol/ideadata/ed52egek/data/bci/data/trainA", "/vol/ideadata/ed52egek/data/bci/data/trainB", "/vol/ideadata/ed52egek/data/bci/data/testA", "out/path", "results.json", config) 
# results will also be saved to "out/path/results.json", all features as tensors to out/path/<model>/*.pt

print(results["inception"]["fid_train"]) # fid with inception features between synthetic and train dataset

2024-09-05 19:28:41,131 | feature_computation.py-compute_features-0134 | INFO | Computing features for model: inception
2024-09-05 19:28:41,133 | feature_computation.py-compute_features-0140 | INFO | Precomputing features of train data
2024-09-05 19:28:41,187 | __init__.py-get_data_from_folder-0063 | INFO | 3396 files found for path /vol/ideadata/ed52egek/data/bci/data/trainA
2024-09-05 19:28:41,188 | feature_computation.py-precompute_features_from_path-0126 | INFO | Precomputed feature tensor already found in: out/path/inception/hashdata_inception_8de8a4e5365fc8f6330167988e0767581a1b0404.pt
2024-09-05 19:28:41,190 | feature_computation.py-compute_features-0143 | INFO | Precomputing features of test data
2024-09-05 19:28:41,241 | __init__.py-get_data_from_folder-0063 | INFO | 3396 files found for path /vol/ideadata/ed52egek/data/bci/data/trainB
2024-09-05 19:28:41,243 | feature_computation.py-precompute_features_from_path-0126 | INFO | Precomputed feature tensor already found in: out/p

# How to use a custom feature extractor and a preloaded dataset

In [7]:
from beyondfid.default_config import config 
from beyondfid.run import run_generic
import torch # only for torch.randn

# a callable object, takes 0-1 normalized (BxCxHxW) -> (BxF) and computes features 
forward_function = lambda x:  torch.randn((x.size()[0], 2048))

train_images = {"file_key_train":torch.zeros((100, 3, 512, 512))} # key will be used to save features as tensor
test_images = {"file_key_test":torch.zeros((100, 3, 512, 512))} 
generated_images = {"file_key_gen":torch.zeros((100, 3, 512, 512))} 

# define metrics you want to compute 
config.metric_list = "fid,kid,prdc" 

# (optional) set metric hyperparams -- see beyondfid/default_config.py for a full list
config.metrics.prdc.nearest_k = 3 


results = run_generic(train_images, test_images, generated_images, forward_function, config, output_path="./.cache/")
# Some metrics like FID, IS, KID, FLD do not need train and test data. Just set the same path twice. 
print(results)

2024-09-05 19:31:25,945 | feature_computation.py-compute_features-0134 | INFO | Computing features for model: generic
2024-09-05 19:31:25,948 | feature_computation.py-compute_features-0140 | INFO | Precomputing features of train data
2024-09-05 19:31:25,950 | feature_computation.py-precompute_features_from_path-0111 | INFO | Computing features for generic and saving to ./.cache/generic/hashdata_generic_file_key_train.pt


100%|██████████| 1/1 [00:02<00:00,  2.45s/it]


2024-09-05 19:31:28,505 | feature_computation.py-compute_features-0143 | INFO | Precomputing features of test data
2024-09-05 19:31:28,507 | feature_computation.py-precompute_features_from_path-0111 | INFO | Computing features for generic and saving to ./.cache/generic/hashdata_generic_file_key_test.pt


100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


2024-09-05 19:31:30,217 | feature_computation.py-compute_features-0146 | INFO | Precomputing features of synth data
2024-09-05 19:31:30,218 | feature_computation.py-precompute_features_from_path-0111 | INFO | Computing features for generic and saving to ./.cache/generic/hashdata_generic_file_key_gen.pt


100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


2024-09-05 19:31:32,060 | run.py-run_generic-0092 | INFO | Computing fid
2024-09-05 19:31:44,025 | fid.py-_compute-0139 | WARNING | FID computation failed due to Imaginary component 1.3304660443738239e+97
2024-09-05 19:31:44,027 | fid.py-_compute-0140 | WARNING | Setting FID to -1
2024-09-05 19:31:51,199 | fid.py-_compute-0139 | WARNING | FID computation failed due to Imaginary component 7.946414803329755e+102
2024-09-05 19:31:51,201 | fid.py-_compute-0140 | WARNING | Setting FID to -1
2024-09-05 19:31:51,202 | run.py-run_generic-0092 | INFO | Computing kid
2024-09-05 19:31:51,316 | run.py-run_generic-0092 | INFO | Computing prdc
{'fid': None, 'kid': None, 'prdc': {'generic': {'prdc_train': {'precision': 0.62, 'recall': 0.61, 'density': 1.1566666666666667, 'coverage': 0.97}, 'prdc_test': {'precision': 0.6, 'recall': 0.7, 'density': 1.1099999999999999, 'coverage': 0.94}}}}
